In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
%matplotlib inline


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



## Class for the computation of the ESN activities

In [2]:
class Echo:
    
    def __init__(self,dt,tau_m,tau_M,diluition,N,W_in):
            
            self.dt=dt
            self.N=N
            self.tau_m=tau_m
            self.tau_M=tau_M
            self.Diluition=diluition
            
            W_np=np.random.uniform(-1,1,[N,N])
            D=np.random.uniform(0,1,(N,N))>np.ones((N,N))*diluition
            W_np=W_np*D.astype(int)
            
            eig=np.linalg.eigvals(W_np)
            self.eig=eig
            
            alpha_np=dt/(2*tau_m)
            pho_np=1-2*tau_m/tau_M
            
            W_np=pho_np*W_np/(np.max(np.absolute(eig)))
            
            self.W_np=W_np
            self.alpha_np=alpha_np
            self.pho_np=pho_np
            
            self.W=tf.Variable(W_np,trainable=False,dtype=tf.float32)
            self.alpha=tf.constant(alpha_np,dtype=tf.float32)
            
            self.W_in_np=W_in
            self.W_in=tf.Variable(W_in,trainable=False,dtype=tf.float32)
            self.N_in=np.shape(W_in)[0]
            
    def evolution_graph(self,T,init_state,inputs):

        
        state_hidden=init_state
        
        states_hidden=[]
        xs=[]
        
        for t in range(T):
            
            prev_state=tf.identity(state_hidden)
            state_hidden=( (1-self.alpha)*prev_state+self.alpha*tf.nn.relu( tf.matmul(prev_state,self.W)+tf.matmul(inputs[:,:,t],self.W_in) )) 
                        
            states_hidden.append(state_hidden)
                        
        states=tf.concat([tf.expand_dims(s,2) for s in states_hidden],2)

    
        return state_hidden, states


## Definition of the input connectivity matrix

The weigths of the input connectivity follow a lognormal distribution. The number of connections to an input node is 
inversely proportional to their value...

In [3]:
def W_input(n_proj,N,n_mean_conn):
    
    m = 1
    v = 0.5
    mu = np.log((m**2)/np.sqrt(v+m**2))
    sigma = np.sqrt(np.log(v/(m**2)+1))
    X= np.random.lognormal(mu, sigma, N)

    alpha=np.mean(np.round(X))*n_mean_conn
    epsilon=0.3
    
    while np.mean(np.round(alpha/X))<n_mean_conn-epsilon or np.mean(np.round(alpha/X))>n_mean_conn+epsilon:

        if  np.mean(np.round(alpha/X))<n_mean_conn-epsilon:
            alpha=alpha+0.01 

        if  np.mean(np.round(alpha/X))>n_mean_conn+epsilon:
            alpha=alpha-0.01


    Y=np.round(alpha/X).astype(int)
    W_In=np.zeros([N,N_proj])

    for i in range(N):
        W_In[i,np.random.randint(0,N_proj,Y[i])]=X[i]

    return W_In, X

## Definition of the input

Specifying N_basis and N_class, the following class creates N_class x N_class x N_basis x Sequence_lenght sequences.
The procedure will generate sequences and labels that will test the memory capacity of the network.
For a description of the procedure, see Appendix of the paper.


In [4]:
class input_stimuli:
    
    def __init__(self,T,N_basis,N_class,T_odour):
        
        import scipy.io
        odours = scipy.io.loadmat('hallem_olsen.mat')
        odours.keys()
        odours=odours['hallem_olsen']
        
        self.odours=odours/(np.max(np.max(odours,0)))                         # Normalisation of the data
        self.N_odours=np.shape(odours)[0]
        
        
        self.T=T
        self.N_class=N_class
                
        Sequence_length=3
        N_basis=N_class*N_basis
        
        
        N_sequence=(Sequence_length*N_class)*N_basis
        self.N_stimuli=N_sequence
        
        Sequence_basis=np.zeros([N_basis,Sequence_length])
        
        vector_index=np.ones([self.N_odours]);
        vector_index=vector_index.astype(int)
        
        # The loop defines, for each output class, a set of sequences (that in the paper we call 'context') composed by elements without repetitions 
        for i in range(N_class):
            
            random_int=np.random.permutation(self.N_odours)
        
            Sequence_basis[i*np.int(N_basis/N_class):(i+1)*np.int(N_basis/N_class),:]=np.reshape(random_int[0:Sequence_length*np.int(N_basis/N_class)],[np.int(N_basis/N_class),Sequence_length])
        
        
        Sequence=np.zeros([N_basis*Sequence_length*N_class,Sequence_length])
        Seq_basis=np.zeros([N_basis*Sequence_length*N_class,Sequence_length])
        
        # It replicates each sequence in the array Sequence_basis N_class*Sequence_length times to apply (later) 'perturbations' 
        # to its elements (see paper)  
        for i in range(np.shape(Sequence_basis)[0]):
            
            Seq_basis[N_class*Sequence_length*i:N_class*Sequence_length*i+N_class*Sequence_length,:]=Sequence_basis[i,:]
        
        Sequence=np.copy(Seq_basis)
        self.Sequence_basis=Seq_basis  
        
        ordered=np.linspace(0,self.N_odours-1,self.N_odours)
        permutated=np.random.permutation(ordered)
    
        # The multiples of N_split define when the perturbations are repeated
        N_split=int(np.shape(Seq_basis)[0]/N_class)
        
        # The loop applies 'perturbations' (change of elements) to the sequences 
        for i in range(int(N_basis/N_class)):
                                    
            
            for j in range(N_class*Sequence_length):
                
                # Index of the element that is changed
                h=np.floor(j/N_class).astype(int)
                
                for l in range(N_class):
                    
                    if l==0:
                        
                        Sequence[N_class*Sequence_length*i+j,h]=permutated[N_class*Sequence_length*i+j]
                    
                    # Repetitions of the perturbation 
                    Sequence[N_class*Sequence_length*i+j+N_split*l,h]=permutated[N_class*Sequence_length*i+j]
                
        Sequence=Sequence.astype(int)
        
        self.Sequence=Sequence        
        
        # Sequence is the array of sequences, it is a dictionary for the multi-d response of the input nodes.
        
        # Each element of each sequence corresponds to a vector in the odours array, containing the response of the input nodes
        # to the stimulus. Such vector is then is expanded for the amount of time T_odour that the NN is subjected to it.
        stimuli=np.zeros([self.N_stimuli,np.shape(odours)[1],Sequence_length*T_odour])
        for i in range(Sequence_length):
            
            stimuli[0:self.N_stimuli,0:np.shape(odours)[1],T_odour*i:T_odour*i+T_odour]=np.tile(np.expand_dims(self.odours[Sequence[:,i],:],2),[1,1,T_odour])
        
        
        self.stimuli=tf.constant(stimuli,dtype=tf.float32)
        
        # Definition of the desired output for the classification
        label=np.zeros([self.N_stimuli,N_class])
        
        # Array that helps to build the labels of the dataset of sequence
        label_basis=np.zeros([N_class,N_class,N_class])
        
        # Starting from an identity matrix, it builds the rest of the labels for  by shifting across the columns
        label_basis[:,:,0]=np.eye(N_class)
        for i in range(N_class-1):
            label_basis[:,:,i+1]=np.concatenate((label_basis[:,i+1:,0],label_basis[:,:i+1,0]),1)
        
        # It repeats the pattern of labels defined above for each set composed by N_split sequences (thus N_split/N_class times) 
        for i in range(N_class):
            label[i*N_split:i*N_split+N_split,:]=np.tile(label_basis[:,:,i],[int(N_split/N_class),1])
         
        self.labels=tf.constant(label,dtype=tf.float32)
        self.labels_np=label
        
        # Now that the labels are defined, we added multiplicative white noise to the data (the realisation of the noise 
        # is different for each sample). This is done in 'next_batch'...
        
    def next_batch(self,rand_ind,N_proj,sigma_noise):
                
        s_=tf.gather_nd(self.stimuli,rand_ind)
        
        s=s_+tf.random_normal(tf.shape(s_),0)*s_*sigma_noise
        
        labels=tf.gather_nd(self.labels,rand_ind)
        
        return s,labels

## Function for the computation of the specificity

The arguments are:
<ul>
    <li> state: the values of activities across the set of sequences considered (state has the dimensionality [Number of seq, Number of nodes]) </li>
    <li> labels: the desired output of the classification (dimensionality [Number of seq, Number of Classes]) </li>
    
The function will compute 2 specificity metrics, based on two different normalisations (see paper)


In [5]:
def specificity(state,labels):
    
    N_class=np.shape(labels)[1]
    N=np.shape(state)[1]
    N_test=np.shape(labels)[0]
    
    # It contains the number of times that a node is active for each class 
    activation_class=np.zeros([N,N_class]) 
    
    # For each separate class, the loop computes the number of times a node is active
    for i in range(N_class):
        
        # Filters the activity with the binary mask np.tile(np.expand_dims(labels[:,i],1),[1,N]) to consider only the sequences 
        # whose desired class is i 
        activity_class=state*np.tile(np.expand_dims(labels[:,i],1),[1,N])
        
        # Summation over the positive values across the first dimension, i.e. different sequences
        activation_class[:,i]=np.transpose(np.sum((activity_class>0),0))
            
    # Tensor describing the specificity
    spec_tensor=np.zeros([N,N_class,N_class])
    spec_tensor1=np.zeros([N,N_class,N_class])
    spec_tensor2=np.zeros([N,N_class,N_class])
    
    # Specificity
    spec=np.zeros([N])
    spec1=np.zeros([N])
    spec2=np.zeros([N])

    # Loop over each pair of classes 
    for i in range(N_class):
        
        for j in range(N_class):
            
            # Index to consider when a node is active at least once for the pair of classes considered
            index_help=activation_class[:,i]+activation_class[:,j]>0
            
            # Specificity, normalised with the total number of times the node was active
            spec_tensor[index_help,i,j]=np.absolute(activation_class[index_help,i]-activation_class[index_help,j])/(activation_class[index_help,i]+activation_class[index_help,j])
            
             # Specificity, normalised with the total number of sequences 
            spec_tensor1[index_help,i,j]=np.absolute(activation_class[index_help,i]-activation_class[index_help,j])/N_test
            

    for i in range(N):
        
        if np.sum(spec_tensor[i,:,:]>0)>0:
            
            # Average over the positive values in the specificity tensors
            spec[i]=np.mean(spec_tensor[i,spec_tensor[i,:,:]>0])
            spec1[i]=np.mean(spec_tensor1[i,spec_tensor[i,:,:]>0])

    return spec1, spec, spec_tensor, activation_class
    

## Tensorflow graph for the training

The following class handles the training of three possible models:

<ul>
    <li> HL: an additional hidden layer on the top of the ESN. If scan=True it is possible to compute the performance as the            number of nodes in the hidden layer or the learning rate change. In the code the arrays N_hidden_values (number of              nodes) and alpha_size_values (learning rate) contain the values that are scanned by the algorithm automatically if              scan=True. Of course, it is possible to change them. </li>
    <li> SpaRCe: training of N_copies NNs starting from different initial conditions, i.e. different starting sparsity level.            The algorithm is composed by an initialisation phase (it initialises the starting value of the thresholds, theta_g)            and a training phase (where the thresholds theta_i and the output weights W_out are optimised). The thresholds are              theta=theta_g+theta_i, where the first factor corresponds to the initialisation, and the second factor is optimised            (this split is due to convenience, to have the possibility to check each factor separately)  </li>
    <li> ESN: Training of the readout of an ESN. It is possible to compute the performance as the learning rate varies, as in            HL. </li>
    
    
    

In [24]:
class Models:
    
    def __init__(self,N,N_class,alpha_size,batch_size,N_ep):
        
        self.N=N
        
        self.N_class=N_class
        
        self.alpha_size=alpha_size
        
        self.batch_size=batch_size
        
        self.N_episodes=N_ep
        
        
    def HL(self,state,y_true,scan,N_copies,scan_learning_rate=True):
        
        R=10*y_true
        N_hidden=100
        
        if scan==False:
            
            alpha_size_values=[self.alpha_size]
            N_hidden_values=[N_hidden]
        
        if scan==True:
            
            if scan_learning_rate==True:
                
                alpha_size_values=np.linspace(0.0002,0.0008,N_copies)
                N_hidden_values=np.int32(np.ones([N_copies])*N_hidden)
                
            else:
                
                alpha_size_values=np.ones([N_copies])*self.alpha_size
                N_hidden_values=np.linspace(10,100,N_copies)
            
            
        W_hidden=[]
        b_hidden=[]
        y1=[]
        W_out=[]
        y2=[]
        error=[]
        train=[]

        for i in range(N_copies):

            W_hidden.append(tf.Variable(np.random.uniform(-1,1,[self.N,N_hidden_values[i]])/(self.N/10),dtype=tf.float32))
            b_hidden.append(tf.Variable(np.random.uniform(-1,1,[N_hidden_values[i]])/(self.N),dtype=tf.float32))

            y1.append(tf.nn.relu(tf.matmul(state,W_hidden[i])+b_hidden[i]))

            W_out.append(tf.Variable(np.random.uniform(-1,1,[N_hidden_values[i],self.N_class])/(self.N/10),dtype=tf.float32))

            y2.append( tf.matmul(y1[i],W_out[i]) )

            error.append(tf.reduce_mean(tf.square(R-y2[i])))

            train.append(tf.train.GradientDescentOptimizer(learning_rate=alpha_size_values[i]).minimize(error[i]))
            
        return y2, error, train 
    
    
    def ESN(self,state,y_true,scan,N_copies):
        
        R=10*y_true
        
        if scan==False:
    
            alpha_size_values=[self.alpha_size]
                               
        
        if scan==True:
            
            alpha_size_values=np.linspace(0.0002,0.0006,N_copies)
            
        W_out=[]
        y=[]
        error=[]
        train=[]

        for i in range(N_copies):

            W_out.append(tf.Variable(np.random.uniform(-1,1,[self.N,self.N_class])/(self.N/10),dtype=tf.float32))

            y.append( tf.matmul(state,W_out[i]) )

            error.append(tf.reduce_mean(tf.square(R-y[i])))

            train.append(tf.train.GradientDescentOptimizer(learning_rate=alpha_size_values[i]).minimize(error[i]))

        return y, error, train 
    
    def SpaRCe(self,state,y_true,theta_g_start):
        
        theta_g=[]
        theta_i=[]
        W_out=[]
        state_sparse=[]
        y=[]
        error=[]
        train1=[]
        train2=[]
        
        R=10*y_true
        
        theta_istart=np.random.randn(N)/N
        
        for i in range(N_copies):

            theta_g.append(tf.Variable(np.expand_dims(theta_g_start[:,i],0), trainable=False, dtype=tf.float32))

            theta_i.append(tf.Variable(theta_istart,dtype=tf.float32))

            W_out.append(tf.Variable(np.random.uniform(0,1,[N,N_class])/(N/10),dtype=tf.float32))

            state_sparse.append(tf.nn.relu(state-theta_g[i]-theta_i[i]))               

            y.append(tf.matmul(state_sparse[i],W_out[i]))

            error.append(tf.reduce_mean(tf.square(R-y[i])))

            train1.append(tf.train.GradientDescentOptimizer(learning_rate=alpha_size).minimize(error[i],var_list=[W_out[i]]))
            train2.append(tf.train.GradientDescentOptimizer(learning_rate=alpha_size/10).minimize(error[i],var_list=[theta_i[i]]))

            train=train1+train2
        
        
        return state_sparse, y, error, train
        



## Training, main cell

In [30]:

T=30                        # Time steps in a sequence
N_class=2                   # Number of output classes
N=1000                      # Number of nodes in the ESN
T_odour=10                  # Temporal lenght of an element in the sequence

dt=0.01                     # time frame
tau_m=0.05                  # Minimum timescale in the ESN
tau_M=2                     # Maximum timescale in the ESN (the timescales of the ESN will be approximatly inside the interval
                            # [tau_m tau_M] )
    
diluition=0.99              # Probability of a zero in the connectivity matrix of the ESN
N_copies=10                 # Number of ESNs to be trained, starting from different initial conditions of the thresholds (theta_g)

N_episodes=100000           # Number of training instances
N_check=20                  # Number of times the code computes the performance, equally spaced across the simulation time
    
rescale_W_in=1              # Multiplicative factor of the input connectivity

N_basis=16
sigma_noise=0.2

inputs=input_stimuli(T,N_basis,N_class,T_odour)
            
N_proj=np.shape(inputs.odours)[1]

# Indexes to test the performance of the network
index_test_=np.arange(0,inputs.N_stimuli,dtype=int)
N_epochs=10
index_test_=np.tile(index_test_,[N_epochs])
index_test=np.zeros([np.shape(index_test_)[0],1])
index_test[:,0]=index_test_

## RNN DEFINITION AND TENSORFLOW GRAPH FOR THE ACTIVITY OF THE ESN AND SPARCE

## RNN DEFINITION

W_in,X=W_input(N_proj,N,n_mean_conn=6)
W_in=W_in.T/rescale_W_in

rnn=Echo(dt,tau_m,tau_M,diluition,N,W_in)

init_state=tf.placeholder(tf.float32,[None,N])
ind_nextbatch=tf.placeholder(tf.int32,[None,1])

stimuli,y_true=inputs.next_batch(ind_nextbatch,N_proj,sigma_noise)

state,states=rnn.evolution_graph(T,init_state,stimuli)


## MODEL DEFINITION
# MODEL=1 -> SpaRCe
# MODEL=2 -> HL, additional NN layer after the ESN
# MODEL=3 -> Simple ESN
# The learning rates and the batch_sizes specified for the different models are good possible values, selected after grid search

MODEL=2

N_copies=10

if MODEL==1:
    
    alpha_size=0.002                                                  # Learning rate
    batch_size=30    
    NN_model=Models(N,N_class,alpha_size,batch_size,N_episodes)
    
    theta_g_start=np.zeros([N,N_copies])
        
    with tf.Session() as sess:
        
        sess.run(tf.global_variables_initializer())
        
        state_=sess.run(state,feed_dict={init_state:np.zeros([np.shape(index_test)[0],N]),ind_nextbatch:index_test})

        for i in range(N_copies):

            theta_g=np.percentile(state_,i*N_copies,0)

            x=(state_-theta_g)>0
            theta_g_start[:,i]=theta_g
    
    state_sparse, y, error, train = NN_model.SpaRCe(state,y_true,theta_g_start)
    
if MODEL==2:
    
    alpha_size=0.0006                                                # Learning rate
    batch_size=30   
    NN_model=Models(N,N_class,alpha_size,batch_size,N_episodes)      
    
    scan=True                                                     
    
    if scan==False:
        N_copies=1
    
    y, error, train = NN_model.HL(state,y_true,scan,N_copies,scan_learning_rate=True)
    

if MODEL==3:
    
    alpha_size=0.0005                                               # Learning rate
    batch_size=100
    
    scan=True
    
    if scan==False:
        N_copies=1
    
    NN_model=Models(N,N_class,alpha_size,batch_size,N_episodes)
    y, error, train = NN_model.ESN(state,y_true,scan,N_copies)
    

init=tf.global_variables_initializer()

with tf.Session() as sess:
    
    sess.run(init)
    
    for n in range(N_episodes+1):

        ind_next=np.random.randint(0,inputs.N_stimuli,[batch_size,1])

        _=sess.run([t for t in train],feed_dict={init_state:np.zeros([batch_size,N]),ind_nextbatch:ind_next})

        if n%(np.round(N_episodes/N_check))==0 and n>0:

            index_help=(n/(np.round(N_episodes/N_check))).astype(int)
            
            for i in range(N_copies):

                matches=tf.equal(tf.argmax(y_true,1),tf.argmax(y[i],1))
                p=tf.reduce_mean(tf.cast(matches,tf.float32))
                
                                
                if MODEL==1:
                    
                    labels_,state_,p_ra,error_=sess.run([y_true,state_sparse[i],p,error[i]],feed_dict={init_state:np.zeros([np.shape(index_test)[0],N]),ind_nextbatch:index_test})
                    
                    coding_level=np.mean(state_>0)
                    spec1, spec, spec_tensor, activation_class=specificity(state_,labels_)
                    print('Iteration: ',n,'SpaRCe ',i,'Probability: ',p_ra,'Error: ',error_,'Coding: ',coding_level,'Spec: ',np.mean(spec),' ',np.mean(spec1))

                    
                else:
                    
                    p_ra,error_=sess.run([p,error[i]],feed_dict={init_state:np.zeros([np.shape(index_test)[0],N]),ind_nextbatch:index_test})

                    if MODEL==2:
                        print('Iteration: ',n,'ESN_HL ',i,'Probability: ',p_ra,'Error: ',error_)
                    if MODEL==3:
                        print('Iteration: ',n,'ESN ',i,'Probability: ',p_ra,'Error: ',error_)
                        




0
[<tf.Operation 'GradientDescent_124' type=NoOp>]
1
[<tf.Operation 'GradientDescent_124' type=NoOp>, <tf.Operation 'GradientDescent_125' type=NoOp>]
2
[<tf.Operation 'GradientDescent_124' type=NoOp>, <tf.Operation 'GradientDescent_125' type=NoOp>, <tf.Operation 'GradientDescent_126' type=NoOp>]
3
[<tf.Operation 'GradientDescent_124' type=NoOp>, <tf.Operation 'GradientDescent_125' type=NoOp>, <tf.Operation 'GradientDescent_126' type=NoOp>, <tf.Operation 'GradientDescent_127' type=NoOp>]
4
[<tf.Operation 'GradientDescent_124' type=NoOp>, <tf.Operation 'GradientDescent_125' type=NoOp>, <tf.Operation 'GradientDescent_126' type=NoOp>, <tf.Operation 'GradientDescent_127' type=NoOp>, <tf.Operation 'GradientDescent_128' type=NoOp>]
5
[<tf.Operation 'GradientDescent_124' type=NoOp>, <tf.Operation 'GradientDescent_125' type=NoOp>, <tf.Operation 'GradientDescent_126' type=NoOp>, <tf.Operation 'GradientDescent_127' type=NoOp>, <tf.Operation 'GradientDescent_128' type=NoOp>, <tf.Operation 'Gradien

KeyboardInterrupt: 